In [3]:
!pip install langchain
!pip install faiss-cpu
!pip install -U sentence-transformers
!pip install -U langchain-openai

  Obtaining dependency information for langchain-core<0.3.0,>=0.2.38 from https://files.pythonhosted.org/packages/bc/02/2b2cf9550cee1a7ffa42fe60c55e2d0e7d397535609b42562611fb40e78d/langchain_core-0.2.41-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 2.0 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.7
    Uninstalling langchain-core-0.3.7:
      Successfully uninstalled langchain-core-0.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-ollama 0.2.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.


## EDA

In [4]:
import pandas as pd

In [5]:
df= pd.read_csv("Symptom2Disease.csv")
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
df.rename(columns={'label': 'label_name', 'text':'text'},inplace=True)

In [8]:
df.head()

,label_name,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [9]:
a = df['label_name'].unique()
print(a)
print(type(a))

['Psoriasis' 'Varicose Veins' 'Typhoid' 'Chicken pox' 'Impetigo' 'Dengue'
 'Fungal infection' 'Common Cold' 'Pneumonia' 'Dimorphic Hemorrhoids'
 'Arthritis' 'Acne' 'Bronchial Asthma' 'Hypertension' 'Migraine'
 'Cervical spondylosis' 'Jaundice' 'Malaria' 'urinary tract infection'
 'allergy' 'gastroesophageal reflux disease' 'drug reaction'
 'peptic ulcer disease' 'diabetes']
<class 'numpy.ndarray'>


In [10]:
# sort target data
target=['Psoriasis', 'Varicose Veins', 'Typhoid', 'Chicken pox',
       'Impetigo', 'Dengue', 'Fungal infection', 'Common Cold',
       'Pneumonia', 'Dimorphic Hemorrhoids', 'Arthritis', 'Acne',
       'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Jaundice', 'Malaria',
       'urinary tract infection', 'allergy',
       'gastroesophageal reflux disease', 'drug reaction',
       'peptic ulcer disease', 'diabetes']
real_target= sorted(target)
real_target

['Acne',
 'Arthritis',
 'Bronchial Asthma',
 'Cervical spondylosis',
 'Chicken pox',
 'Common Cold',
 'Dengue',
 'Dimorphic Hemorrhoids',
 'Fungal infection',
 'Hypertension',
 'Impetigo',
 'Jaundice',
 'Malaria',
 'Migraine',
 'Pneumonia',
 'Psoriasis',
 'Typhoid',
 'Varicose Veins',
 'allergy',
 'diabetes',
 'drug reaction',
 'gastroesophageal reflux disease',
 'peptic ulcer disease',
 'urinary tract infection']

## Data ingestion

In [12]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the CSV document
loader = CSVLoader(file_path="Symptom2Disease.csv")
documents = loader.load()
# Split documents into smaller chunks (optional)
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Use the documents in your LangChain application
#print(docs)  # Output: List of Document objects


In [13]:
docs[0]

Document(metadata={'source': 'Symptom2Disease.csv', 'row': 0}, page_content=': 0\nlabel: Psoriasis\ntext: I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.')

In [77]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings=(
    OllamaEmbeddings(model="llama3.2:1b") #llama3.2:1b #llama3.1
)
# Refrence doc : https://ollama.com/blog/embedding-models

#db=FAISS.from_documents(docs,embeddings)
#db # Takes lot of time my laptop is not able to process it 


'\nfrom langchain_community.embeddings import OllamaEmbeddings\n\nembeddings=(\n    OllamaEmbeddings(model="llama3.2:1b") #llama3.2:1b #llama3.1\n)\n'

In [9]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama3.2:1b', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/k0/r9q8frbj00n77npfmkp_j3xm0000gn/T/ipykernel_17615/3419084168.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/piyushprabhakar/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-02 17:55:15.470303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CP

In [48]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## FAISS

In [15]:
from langchain_community.vectorstores import FAISS

In [16]:
db=FAISS.from_documents(docs,embeddings)
db # Takes less time with hugging face embedding # System hang with Olama embedding

In [17]:
### querying 
query="My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
docs=db.similarity_search(query)
docs[0].page_content

': 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [18]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

': 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [19]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'Symptom2Disease.csv', 'row': 1}, page_content=': 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'),
  0.2501623),
 (Document(metadata={'source': 'Symptom2Disease.csv', 'row': 22}, page_content=': 22\nlabel: Psoriasis\ntext: My skin is peeling, particularly on my knees, elbows, and scalp. This peeling is frequently accompanied by a stinging or burning feeling.'),
  0.287296),
 (Document(metadata={'source': 'Symptom2Disease.csv', 'row': 39}, page_content=': 39\nlabel: Psoriasis\ntext: My skin is peeling in places, especially on my knees, elbows, and arms. This peeling is often accompanied by a painful or burning sensation. I am also developing small dents on my nails, which is really concerning.'),
  0.40929204),
 (Document(metadata={'source': 'Symptom2Disease.csv', 'row': 8}, page_content=': 8\nlabel: Psoriasis\ntext: I have noticed a sudd

In [20]:
embedding_vector=embeddings.embed_query(query)
#embedding_vector

In [21]:
docs_score=db.similarity_search_by_vector(embedding_vector)
#docs_score

In [22]:
### Saving And Loading
db.save_local("faiss_index")

In [24]:
local_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=local_db.similarity_search(query)
docs

[Document(metadata={'source': 'Symptom2Disease.csv', 'row': 1}, page_content=': 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'),
 Document(metadata={'source': 'Symptom2Disease.csv', 'row': 22}, page_content=': 22\nlabel: Psoriasis\ntext: My skin is peeling, particularly on my knees, elbows, and scalp. This peeling is frequently accompanied by a stinging or burning feeling.'),
 Document(metadata={'source': 'Symptom2Disease.csv', 'row': 39}, page_content=': 39\nlabel: Psoriasis\ntext: My skin is peeling in places, especially on my knees, elbows, and arms. This peeling is often accompanied by a painful or burning sensation. I am also developing small dents on my nails, which is really concerning.'),
 Document(metadata={'source': 'Symptom2Disease.csv', 'row': 8}, page_content=': 8\nlabel: Psoriasis\ntext: I have noticed a sudden peeling of skin at different parts of my bo

In [25]:
docs[0]

Document(metadata={'source': 'Symptom2Disease.csv', 'row': 1}, page_content=': 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.')

### Intent Classification

In [151]:
class TopicClassifier:
    def __init__(self, llm):
        self.llm = llm
        self.topics = real_target

    def classify(self, query):
        prompt = f"Classify the following question into one of these topics: '{','.join(self.topics)}': '{query}'"
        response = self.llm.predict(text=prompt, max_tokens=500)
        topic = response.strip().lower()
        return topic

### Using GPT-3.5 as LLM

In [38]:
import getpass
api_key = getpass.getpass("Enter the OPENAI_API_KEY key: ")


Enter the OPENAI_API_KEY key:  ········


In [116]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key,
                  model_name="gpt-3.5-turbo",
                  temperature=0.0)

In [83]:
classification = TopicClassifier(llm)

In [84]:
query = "The veins on my legs are very noticeable and cause me discomfort. It seems like there is a major bruise "#and I get cramps when I run.
answer = classification.classify(query)
print(answer)

varicose veins


In [85]:
answer = classification.classify("No energy, no appetite, sick, red spots.")
print(answer)

chicken pox


In [87]:
answer = classification.classify("suffering from severe body pain, a headache, and vomiting. Red spots have covered my entire body, causing intense itching.")
print(answer) #dengue-"I'm suffering from severe body pain, a headache, and vomiting. Red spots have covered my entire body, causing intense itching."

chicken pox


In [89]:
answer = classification.classify("i have had a rash on my skin that looks like dischromic patches, and I also have a lot of nodular eruptions and really bad itching. During the previous week, it has become worse.")
print(answer)

fungal infection


In [91]:
answer = classification.classify("rash with discolored patches and numerous bumps and its eaching")
print(answer) #Common cold

this question falls under the topic of 'fungal infection'.


### Using Ollama

In [76]:
from langchain_ollama.chat_models import ChatOllama
ollamaLLM = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)


In [77]:
ollamaLLM

ChatOllama(model='llama3.2', temperature=0.0)

In [92]:
classification = TopicClassifier(ollamaLLM)

In [80]:
query = "The veins on my legs are very noticeable and cause me discomfort. It seems like there is a major bruise "#and I get cramps when I run.
answer = classification.classify(query)
print(answer) #takes very long time

i would classify the question into the topic of: "varicose veins" or more specifically, "vein problems" or "leg vein issues".


In [81]:
answer = classification.classify("No energy, no appetite, sick, red spots.")
print(answer) #takes very long time

the question can be classified under the topic of "symptoms" or more specifically, "common symptoms". it is a general description of symptoms that could be associated with various medical conditions.


In [93]:
answer = classification.classify("suffering from severe body pain, a headache, and vomiting. Red spots have covered my entire body, causing intense itching.")
print(answer)

based on the symptoms described (severe body pain, headache, vomiting, red spots covering the entire body with intense itching), i would classify this question into the following topics:

1. dermatology (skin conditions): the presence of red spots covering the entire body and intense itching suggests a skin condition.
2. allergy: the intense itching could be related to an allergic reaction.

more specifically, the symptoms described are consistent with a severe case of impetigo, which is a bacterial infection that causes red sores on the skin, often accompanied by intense itching and discomfort.


In [94]:
answer = classification.classify("rash with discolored patches and numerous bumps and its eaching")
print(answer) #Common cold

i would classify the question into the following topics:

1. skin conditions:
 * rash
 * discolored patches
 * bumps
 * impetigo (implied by "bumps")
2. infections:
 * fungal infection (implied by "rash with discolored patches and numerous bumps")
3. allergic reactions:
 * allergy (implied by the presence of a rash)
4. general medical conditions:
 * hypertension (not directly related, but could be a contributing factor)

however, without more specific information about the nature of the rash or the "bumps", it's difficult to pinpoint an exact diagnosis.


## Using Prompt

In [95]:
df.head()

,label_name,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [99]:
df.tail()

,label_name,text
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


In [103]:
newData_frame = pd.concat([df.head(), df.tail()])


In [104]:
newData_frame

,label_name,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


In [129]:
questionAnsDict = {}

for index, row in newData_frame.iterrows():
       print(index, row['label_name'], row['text'])


{'Psoriasis': 'My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms.',
 'diabetes': "I constantly sneeze and have a dry cough. My infections don't seem to be healing, and I have palpitations. My throat does ache occasionally, but it usually gets better."}

In [142]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(newData_frame, page_content_column="text")
contextDoc = loader.load()
contextDoc

[Document(metadata={'label_name': 'Psoriasis'}, page_content='I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.'),
 Document(metadata={'label_name': 'Psoriasis'}, page_content='My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'),
 Document(metadata={'label_name': 'Psoriasis'}, page_content='I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints.'),
 Document(metadata={'label_name': 'Psoriasis'}, page_content='There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them.'),
 Document(metadata={'label_name': 'Psoriasis'}, page_content='My nails have small dents or pits in them, and they often feel inflammatory and ten

In [163]:
contextDoc[0]

Document(metadata={'label_name': 'Psoriasis'}, page_content='I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.')

In [139]:
from langchain import PromptTemplate

template = """
You are a medical chatbot. These Human will ask you a questions about their symptoms. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 1-2 word and concise.

For Example
Context: 1\label_name: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'
Question: "Red, itchy rash with dry patches on arms, legs, and torso."
Answer: Psoriasis


Context: {context}
Question: {question}
Answer: 

"""

prompt_1 = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [164]:
rag_chain_0 = (
  {"context": "abc",  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [140]:
result = rag_chain_0.invoke("I constantly sneeze and have a dry cough. My infections don't seem to be healing, and I have palpitations. My throat does ache occasionally, but it usually gets better.")
print(result)

NameError: name 'rag_chain_0' is not defined

## RAG Implementation

### Open AI LLM

In [144]:
from langchain import PromptTemplate

template = """
You are a medical chatbot. These Human will ask you a questions about their symptoms. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 1-2 word and concise.

For Example
Context: 1\nlabel: Psoriasis\ntext: My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.'
Question: "Red, itchy rash with dry patches on arms, legs, and torso."
Answer: Psoriasis


Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [119]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": db.as_retriever(),  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [120]:
result = rag_chain.invoke("No energy, no appetite, sick, red spots.")
print(result)

Chicken pox


In [121]:
result = rag_chain.invoke("The veins on my legs are very noticeable and cause me discomfort. It seems like there is a major bruise ")
print(result)

Varicose Veins


In [122]:
result = rag_chain.invoke("suffering from severe body pain, a headache, and vomiting. Red spots have covered my entire body, causing intense itching.")
print(result)

Dengue


In [124]:
result = rag_chain.invoke("rash with discolored patches and numerous bumps and its eaching")
print(result)

Fungal infection


### Ollama LLM

In [148]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain_ollama = (
  {"context": db.as_retriever(),  "question": RunnablePassthrough()} 
  | prompt 
  | ollamaLLM
  | StrOutputParser() 
)

In [149]:
result = rag_chain_ollama.invoke("No energy, no appetite, sick, red spots.")
print(result)#Takes 1 min

I'm ready to help. What's your question about symptoms?


In [150]:
result = rag_chain_ollama.invoke("suffering from severe body pain, a headache, and vomiting. Red spots have covered my entire body, causing intense itching.")
print(result)

I'm ready to help. What's your question?
